# RAG

#### 1. 필수 라이브러리 설치
faiss-cpu : Faiss Vector DB 

tiktoken : 토큰 수 계산

In [13]:
!pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.1 MB/s eta 0:00:00


#### 2. OpenAI API Key 설정

In [2]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-*******************************************************"

## Retrieval Chain

Vector Databse에 저장되어 있는 데이터를 Retrieval (검색) 하여 가져오는 체인

In [14]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#Faiss Vector Database에 "harrison worked at kensho" 라는 문장을 텍스트로 저장함
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)

#Vector DB를 검색기에 연결
retriever = vectorstore.as_retriever()

#Prompt Template 생성
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

#모델 설정
model = ChatOpenAI()

# Prompt에 들어가는 변수 지정
# 검색대상 : "context"          /  질문 : Input Value
# String 데이터 타입으로 Output 반환
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [16]:
#Prompt Template 설정
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

#context는 직접적인 질문 또는 vector DB 검색기가 될 수 있음  (질문에 답변이 있을 수 있음)
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "korean"})

'해리슨은 켄쇼에서 일했습니다.'

## Conversational Retrieval Chain

대화 형식을 통해 검색기를 사용하는 Chain

In [17]:
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel
from langchain.prompts.prompt import PromptTemplate

#Prompt Template 생성
#질문 Template 생성
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

#답변 Template 생성
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

#문서 Template 생성
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

#여러 문서를 결합하는 함수 생성
def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    #여러 docs의 내용을 기반으로, 문서 별로 문서 Prompt 실행
    doc_strings = [format_document(doc, document_prompt) for doc in docs]

    #여러 문서를 '\n\n' 구분 기호로 구분하여 하나의 문서로 결합
    return document_separator.join(doc_strings)

#Input 체인 생성
#assign을 활용하여 변수 미리 지정
#Buffer를 활용하여 Chat History 생성
#질문 Template 실행 => 모델 실행
_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)

#답변 Template의 변수 지정
#standalone_question이라는 변수의 값을 vector db 검색기와 docs의 context에서 찾음
#찾은 standalone_question 값을 질문으로 반환
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}

#QA Chain으로 연결
#input Chain 결과값을 context에 적용하고, context의 변수를 answer 프롬프트에 적용하여 해당 프롬프트로 모델을 실행함
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [18]:
# input prompt에 질문 전달
conversational_qa_chain.invoke(
    {
        "question": "where did harrison work?",
        "chat_history": [],
    }
)

AIMessage(content='Harrison worked at Kensho.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [19]:
#채팅 히스토리에 데이터를 추가하여 retriever와 함께 context로 사용
conversational_qa_chain.invoke(
    {
        "question": "where did he work?",
        "chat_history": [
            HumanMessage(content="Who wrote this notebook?"),
            AIMessage(content="Harrison"),
        ],
    }
)

AIMessage(content='Harrison worked at Kensho.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

## **With Memory and returning source documents**

메모리에 대화 내용을 저장하고 답변에 포함된 문서의 직접적인 소스를 반환함
=> 결과 값이 어디에서 도출되었는지 확인할 수 있다.

In [20]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory

#대화 내용을 기억할 수 있도록 Memory 설정
#question / answer 형태의 메세지로 memory 반환
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

# First we add a step to load memory
# This adds a "memory" key to the input object
# chat_history의 값을 미리 할당합니다. 이때 history의 값을 가지고 옵니다.
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

# Now we calculate the standalone question
# standalone_question은 qustion과 chat_history로, qustion은 질문, chat_history는 과거의 메모리로 부터 가져옵니다.
# CONDENSE_QUSTION_PROMPT의 standalone_question 부분에 해당 값이 할당되어 모델에 적용됩니다.
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}

# Now we retrieve the documents
# standalone_question 의 값과 vector db 검색기를 문서로 설정합니다.
# 메모리의 chat_history 값이 질문과 함께 문서로 사용됩니다.
# 질문은 standalone_question에서 받아옵니다.
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

# Now we construct the inputs for the final prompt
# 새로운 문서 값을 context로 사용하여 질문에 답변을 할 수 있도록 input 값으로 지정합니다.
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}

# And finally, we do the part that returns the answers
# 결과 프롬프트에 최종 입력값을 집어 넣어 모델을 실행합니다.
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}

# And now we put it all together!
# Chain을 생성합니다.
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

# Chain을 실행합니다.
inputs = {"question": "where did harrison work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Harrison worked at Kensho.', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 'docs': [Document(page_content='harrison worked at kensho')]}

In [21]:
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
# 메모리를 저장합니다.
memory.save_context(inputs, {"answer": result["answer"].content})

In [22]:
#저장된 메모리를 불러옵니다.
memory.load_memory_variables({})

{'history': [HumanMessage(content='where did harrison work?'),
  AIMessage(content='Harrison worked at Kensho.')]}

In [23]:
inputs = {"question": "but where did he really work?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Harrison really worked at Kensho.', response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 'docs': [Document(page_content='harrison worked at kensho')]}

In [24]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [25]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='where did harrison work?'),
  AIMessage(content='Harrison worked at Kensho.'),
  HumanMessage(content='but where did he really work?'),
  AIMessage(content='Harrison really worked at Kensho.')]}